[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1NF6pDYVgjrb02dsIEcZAW4JVsiUKPXFB#scrollTo=xIWQ1hWAu25g)


# CRAWLING BERITA

## Rahma Nurhaliza (210411100176)

Teknik crawling adalah proses pengumpulan informasi dari berbagai sumber di internet secara otomatis. Web crawling biasanya dilakukan
oleh program yang disebut web crawler atau spider. Tujuan dari web crawling adalah untuk mengindeks konten di web sehingga dapat dianalisis.



## Install Library

In [1]:
!pip install requests
!pip install beautifulsoup4
!pip install tqdm

## Import Library

- request digunakan untuk meminta respon pada situs yang akan dilakukan crawling data
- BeatifulSoup digunakan untuk melakukan ekstraksi halaman konten agar dapat mengambil isi-isi konten pada berita di website
- pandas digunakan untuk manipulasi dan analisis data. Memungkinkan untuk bekerja secara tabuler
- tqdm digunakan untuk menampilkan progress bar saat menjalankan loop atau iterasi yang memerlukan waktu lama, sangat berguna ketika ingin memantau seberapa jauh proses iterasi telah berjalan,

In [1]:
from os import replace
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

## CRAWLING BERITA CNN

Data berita berasal dari situs CNN


In [12]:
def cnnnews(page, limit=10):  # Tambahkan parameter limit dengan default 10
    data = {'judul': [], 'berita': [], 'tanggal': [], 'kategori': []}
    article_count = 0  # Menyimpan jumlah artikel yang sudah diambil

    for i in tqdm(range(1, page + 1)):
        if article_count >= limit:
            break  # Hentikan loop jika jumlah artikel sudah mencapai limit

        url = "https://www.cnnindonesia.com/indeks/"
        r = requests.get(url)
        request = r.content
        soup = BeautifulSoup(request, 'html.parser')
        soup = soup.find('div', {'class': 'flex flex-col gap-5'})
        news = soup.findAll('article', {'class': 'flex-grow'})

        for new in tqdm(news):
            if article_count >= limit:
                break  # Menghentikan loop jika jumlah artikel sudah mencapai limit

            a_element = new.find('a', {'aria-label': 'link description'})['href']
            detail_request = requests.get(a_element)
            detail_soup = BeautifulSoup(detail_request.content, 'html.parser')

            # Ambil judul
            judul = detail_soup.find('h1', {'class': 'leading-9'})
            judul = judul.text.strip() if judul else 'No title'

            # Ambil berita
            berita_div = detail_soup.find('div', {'class': 'detail-text'})
            berita = berita_div.text.strip() if berita_div else 'No content'

            # Ambil tanggal
            tanggal = detail_soup.find('div', {'class': 'text-cnn_grey'})
            tanggal = tanggal.text.strip() if tanggal else 'No date'

            # Ambil kategori dari breadcrumb
            breadcrumb_items = detail_soup.find_all('a', {'class': 'text-cnn_black_light3'})
            kategori = breadcrumb_items[1].text.strip() if len(breadcrumb_items) > 1 else 'No category'

            # Bersihkan teks berita dari noise dan kata-kata yang tidak relevan
            noise = detail_soup.find('strong').text.strip() if detail_soup.find('strong') else ''
            berita = berita.replace("ADVERTISEMENT", "").replace("SCROLL TO CONTINUE WITH CONTENT", "").replace("Lihat Juga :", "").replace("Bagikan", "").replace(noise, "").replace("-- ", "").replace("--", "")
            berita = ' '.join(berita.split())  # Hilangkan spasi berlebih

            # Tambahkan data ke list
            data["judul"].append(judul)
            data["berita"].append(berita)
            data["tanggal"].append(tanggal)
            data["kategori"].append(kategori)

            article_count += 1  # Penghitung artikel

    # Simpan data ke dalam CSV
    df = pd.DataFrame(data)
    df.to_csv("berita_cnn.csv", index=False)

    return df



In [13]:
cnnnews(page=1, limit=10)

100%|██████████| 1/1 [00:10<00:00, 10.19s/it]


,judul,berita,tanggal,kategori
0,Pin Garuda Laris di Paris dan 'Tiket' Ajaib di...,Saling tukar pin atau bros sudah jadi budaya d...,"Kamis, 05 Sep 2024 00:55 WIB",Olahraga
1,Viral Pendakwah Pakistan Diberi Miras di Pinra...,Beredar sebuah video merekam sejumlah warga Ka...,"Kamis, 05 Sep 2024 00:36 WIB",Nasional
2,Sejoli di Jakbar Jadi Korban Penyiraman Air Ke...,Pasangan muda-mudi jadi korban aksi penyiraman...,"Kamis, 05 Sep 2024 00:14 WIB",Nasional
3,"Disikat Malaysia, Pelatih Filipina Pasrah pada...",Pelatih Filipina Norman Fegidero menilai anak ...,"Kamis, 05 Sep 2024 00:04 WIB",Olahraga
4,VIDEO: Paus Fransiskus Pakai Pancasila dalam P...,Paus Fransiskus menyinggung soal dasar negara ...,"Rabu, 04 Sep 2024 23:40 WIB",Internasional
5,Bulog Siap Bantu Pengusaha Perempuan Lakukan I...,Perum Bulog mendukung acara Indonesia Womenpre...,"Rabu, 04 Sep 2024 23:32 WIB",Ekonomi
6,Pelatih Malaysia Ungkap Derita Usai Harimau Ma...,Pelatih timnas Malaysia Pau Marti Vicente memb...,"Rabu, 04 Sep 2024 23:29 WIB",Olahraga
7,Dugaan Motif Bunuh Diri Anggota Polsek Ipda BS...,"Kapolres Kulon Progo, AKBP Wilson Bugner Pasar...","Rabu, 04 Sep 2024 23:20 WIB",Nasional
8,Momen Luhut Cium Tangan Paus Fransiskus di Dep...,Menteri Koordinator Bidang Kemaritiman dan Inv...,"Rabu, 04 Sep 2024 22:58 WIB",Internasional
9,Pengelola GBK Tak Tutup Rumput Saat Misa Agung...,Pusat Pengelola Komplek Gelora Bung Karno (PPK...,"Rabu, 04 Sep 2024 22:44 WIB",Olahraga


## CRAWLING BERITA KOMPAS


Data berita berasal dari situs kompas, pada kategori IKN



In [2]:
def kompasnews(page, limit=15): #page: jumlh halaman yg ingin di crawl, limit: batas jumlah berita yang ingin diambil
    data = {'judul': [], 'berita': [], 'tanggal': [], 'kategori':[]} #menyimpan informasi yang akan diambil
    article_count = 0 #menghitung berapa banyak artikel yang telah diambil

    for i in tqdm(range(1, page + 1)):
        if article_count >= limit:
            break

        url = f"https://IKN.kompas.com/?source=navbar&page={i}" #(i) : nomor halaman saat ini, yaitu halaman yang akan dikunjungi crawler.
        r = requests.get(url)
        soup = BeautifulSoup(r.content, 'html.parser')

        # Menemukan elemen artikel
        articles = soup.find_all('div', {'class': 'articleItem'})
        if not articles:
            print(f" menemukan 0 artikel pada halaman {i}")
            continue

        for article in articles:
            if article_count >= limit:
                break

            # Ambil URL artikel
            a_element = article.find('a', {'class': 'article-link'})['href']
            detail_request = requests.get(a_element)
            detail_soup = BeautifulSoup(detail_request.content, 'html.parser')

            # Ambil judul
            judul = article.find('h2', {'class': 'articleTitle'})
            judul = judul.text.strip() if judul else 'No title'

            # Ambil tanggal
            tanggal = article.find('div', {'class': 'articlePost-date'})
            tanggal = tanggal.text.strip() if tanggal else 'No date'

            # Ambil tanggal
            breadcrumb_items = detail_soup.find_all('li', {'class': 'breadcrumb__item'})
            kategori = breadcrumb_items[1].get_text(strip=True) if breadcrumb_items else 'No category'


            # Ambil berita dari artikel detail
            berita_div = detail_soup.find('div', {'class': 'read__content'})
            berita = berita_div.text.strip() if berita_div else 'No content'

            # Tambahkan ke data
            data["judul"].append(judul)
            data["berita"].append(berita)
            data["tanggal"].append(tanggal)
            data["kategori"].append(kategori)

            article_count += 1

    df = pd.DataFrame(data)
    df.to_csv('berita-kompas-IKN.csv', index=False)
    return df

In [3]:
kompasnews(1,limit=10)

100%|██████████| 1/1 [00:16<00:00, 16.81s/it]


,judul,berita,tanggal,kategori
0,2 Proyek KPBU Permukiman Kawasan IKN Tengah Di...,"NUSANTARA, KOMPAS.com - Kementerian Pekerjaan ...",04/09/2024,Ikn
1,"Gratis, Masyarakat dan Tamu VIP yang Minta Dik...","NUSANTARA, KOMPAS.com - Direktorat Lalu Lintas...",04/09/2024,Ikn
2,Kehadiran IKN Dorong Investasi di Balikpapan M...,"NUSANTARA, KOMPAS.com - Aktivitas pembangunan ...",04/09/2024,Ikn
3,"Angka Stunting IKN di Atas Nasional, Ini Upaya...","NUSANTARA, KOMPAS.com - Angka stunting di wila...",04/09/2024,Ikn
4,"Akhirnya, Investor Asing Masuk IKN, Benamkan M...","NUSANTARA, KOMPAS.com - Kabar baik untuk pemba...",04/09/2024,Ikn
5,"IKN Butuh 200.000 Pekerja Konstruksi, ""Mandor ...","NUSANTARA, KOMPAS.com - Pada puncak pembanguna...",04/09/2024,Ikn
6,"Bukan di Kompleks Parlemen, DPR Rapat Bareng O...","NUSANTARA, KOMPAS.com - Komisi II DPR RI mengg...",03/09/2024,Ikn
7,Wali Kota Balikpapan Ajak ASN Naik Transportas...,"NUSANTARA, KOMPAS.com - Wali Kota Balikpapan R...",03/09/2024,Ikn
8,"Angka Stunting di IKN 30 Persen, di Atas Rata-...","NUSANTARA, KOMPAS.com - Angka stunting di wila...",03/09/2024,Ikn
9,Pj Gubernur Kaltim Minta Anggota DPRD 2024-202...,"NUSANTARA, KOMPAS.com - Penjabat (Pj) Gubernur...",03/09/2024,Ikn


Data dari situs kompas kategori Edukasi

In [4]:
def kompasnews(page, limit=15): #page: jumlh halaman yg ingin di crawl, limit: batas jumlah berita yang ingin diambil
    data = {'judul': [], 'berita': [], 'tanggal': [], 'kategori':[]} #menyimpan informasi yang akan diambil
    article_count = 0 #menghitung berapa banyak artikel yang telah diambil

    for i in tqdm(range(1, page + 1)):
        if article_count >= limit:
            break

        url = f"https://edukasi.kompas.com/?source=navbar&page={i}" #(i) : nomor halaman saat ini, yaitu halaman yang akan dikunjungi crawler.
        r = requests.get(url)
        soup = BeautifulSoup(r.content, 'html.parser')

        # Menemukan elemen artikel
        articles = soup.find_all('div', {'class': 'articleItem'})
        if not articles:
            print(f" menemukan 0 artikel pada halaman {i}")
            continue

        for article in articles:
            if article_count >= limit:
                break

            # Ambil URL artikel
            a_element = article.find('a', {'class': 'article-link'})['href']
            detail_request = requests.get(a_element)
            detail_soup = BeautifulSoup(detail_request.content, 'html.parser')

            # Ambil judul
            judul = article.find('h2', {'class': 'articleTitle'})
            judul = judul.text.strip() if judul else 'No title'

            # Ambil tanggal
            tanggal = article.find('div', {'class': 'articlePost-date'})
            tanggal = tanggal.text.strip() if tanggal else 'No date'

            # Ambil tanggal
            breadcrumb_items = detail_soup.find_all('li', {'class': 'breadcrumb__item'})
            kategori = breadcrumb_items[1].get_text(strip=True) if breadcrumb_items else 'No category'


            # Ambil berita dari artikel detail
            berita_div = detail_soup.find('div', {'class': 'read__content'})
            berita = berita_div.text.strip() if berita_div else 'No content'

            # Tambahkan ke data
            data["judul"].append(judul)
            data["berita"].append(berita)
            data["tanggal"].append(tanggal)
            data["kategori"].append(kategori)

            article_count += 1

    df = pd.DataFrame(data)
    df.to_csv('berita-kompas-Edukasi.csv', index=False)
    return df

In [5]:
kompasnews(1,limit=10)

100%|██████████| 1/1 [00:11<00:00, 11.36s/it]


,judul,berita,tanggal,kategori
0,Ingin Jadi Kepsek atau Pengawas? Cek Syarat Da...,KOMPAS.com - Pendaftaran Pendidikan Guru Pengg...,04/09/2024,Edu
1,DPR Dukung Usulan Pembentukan Komite Percepata...,KOMPAS.com - Wakil Ketua Komisi X DPR RI Dede ...,04/09/2024,Edu
2,"H-2 Tembus 2,7 Juta Pelamar, Ini 20 Kementeria...",KOMPAS.com - H-2 menuju penutupan pendaftaran ...,04/09/2024,Edu
3,Pendaftaran KJMU Tahap 2 Tahun 2024 Diperpanja...,KOMPAS.com - Pendaftaran Kartu Jakarta Mahasis...,04/09/2024,Edu
4,"Beasiswa Bill Gates 2025 Dibuka, Kuliah S2-S3 ...",KOMPAS.com - Beasiswa Bill Gates 2025 sudah di...,04/09/2024,Edu
5,Ini 20 Gelar S1 dan S2 Paling Dicari di Masa D...,KOMPAS.com - Salah satu faktor mencapai kesuks...,04/09/2024,Edu
6,Ingin Jadi Kepsek atau Pengawas? Cek Syarat Da...,KOMPAS.com - Pendaftaran Pendidikan Guru Pengg...,04/09/2024,Edu
7,"Disdik DKI: 4.000 Orang Daftar KKI 2024, Kini ...",KOMPAS.com - Sebanyak 4.000 orang mendaftar re...,04/09/2024,Edu
8,Sekolah Semesta Semarang Raih 4 Medali Olimpia...,"KOMPAS.com - Empat Sekolah Semesta, Semarang, ...",04/09/2024,Edu
9,"Pengertian Brainware, Fungsi, dan Jenis-jenisnya",KOMPAS.com - Pengguna komputer disebut juga br...,04/09/2024,Skola


Menggabungkan file berita dari kategori yang berbeda

In [6]:
df_ikn = pd.read_csv('berita-kompas-IKN.csv')
df_edukasi = pd.read_csv('berita-kompas-Edukasi.csv')


# Menggabungkan dua DataFrame
df_combined = pd.concat([df_edukasi, df_ikn])

# Menyimpan hasil gabungan ke file baru
df_combined.to_csv('berita-kompas-combined.csv', index=False)
df_combined


,judul,berita,tanggal,kategori
0,Ingin Jadi Kepsek atau Pengawas? Cek Syarat Da...,KOMPAS.com - Pendaftaran Pendidikan Guru Pengg...,04/09/2024,Edu
1,DPR Dukung Usulan Pembentukan Komite Percepata...,KOMPAS.com - Wakil Ketua Komisi X DPR RI Dede ...,04/09/2024,Edu
2,"H-2 Tembus 2,7 Juta Pelamar, Ini 20 Kementeria...",KOMPAS.com - H-2 menuju penutupan pendaftaran ...,04/09/2024,Edu
3,Pendaftaran KJMU Tahap 2 Tahun 2024 Diperpanja...,KOMPAS.com - Pendaftaran Kartu Jakarta Mahasis...,04/09/2024,Edu
4,"Beasiswa Bill Gates 2025 Dibuka, Kuliah S2-S3 ...",KOMPAS.com - Beasiswa Bill Gates 2025 sudah di...,04/09/2024,Edu
5,Ini 20 Gelar S1 dan S2 Paling Dicari di Masa D...,KOMPAS.com - Salah satu faktor mencapai kesuks...,04/09/2024,Edu
6,Ingin Jadi Kepsek atau Pengawas? Cek Syarat Da...,KOMPAS.com - Pendaftaran Pendidikan Guru Pengg...,04/09/2024,Edu
7,"Disdik DKI: 4.000 Orang Daftar KKI 2024, Kini ...",KOMPAS.com - Sebanyak 4.000 orang mendaftar re...,04/09/2024,Edu
8,Sekolah Semesta Semarang Raih 4 Medali Olimpia...,"KOMPAS.com - Empat Sekolah Semesta, Semarang, ...",04/09/2024,Edu
9,"Pengertian Brainware, Fungsi, dan Jenis-jenisnya",KOMPAS.com - Pengguna komputer disebut juga br...,04/09/2024,Skola
